# 15.2.5 Backpropagation through time

**Backpropagation through time (BPTT):** $\;$ backprop aplicado al grafo computacional de una RNN **desenrrollado (unrolled)** en el tiempo con el fin de hallar un MLE de sus parámetros
$$\mathbf{\Theta}^*=\operatorname*{argmax}_{\mathbf{\Theta}}\;p(\boldsymbol{y}_{1:T}\mid\boldsymbol{x}_{1:T},\mathbf{\Theta})$$

**Modelo y pérdida:**
<div style="text-align:left;">
<table style="text-align:left;"><tr>
<td style="border:none; text-align:left; vertical-align:top;" width=450>

Modelo con logits de salida $\boldsymbol{o}_t$ y sin sesgos por simplicidad:
$$\begin{align*}
  \boldsymbol{h}_t&=\mathbf{W}_{hx}\boldsymbol{x}_t+\mathbf{W}_{hh}\boldsymbol{h}_{t-1}\\%
  \boldsymbol{o}_t&=\mathbf{W}_{ho}\boldsymbol{h}_t%
\end{align*}$$

Pérdida con etiquetas de salida $y_t$:
$$L=\frac{1}{T}\sum_{t=1}^T\ell(y_t,\boldsymbol{o}_t)$$

</td><td style="border:none;" width=50></td>
<td style="border: none;"><img src="Figure_15.9.png" width=550/></td>
</tr></table>
</div>

**Cálculo de gradientes:** $\displaystyle\quad\frac{\partial L}{\partial\mathbf{W}_{hx}},\quad\frac{\partial L}{\partial\mathbf{W}_{hh}},\quad\frac{\partial L}{\partial\mathbf{W}_{ho}}$
* El último es fácil de calcular ya que es local a cada $t$
* Los dos primeros dependen del estado oculto, por lo que requieren mirar atrás en el tiempo...
* Con $\mathbf{W}_h$ igual a $\mathbf{W}_{hh}$ y $\mathbf{W}_{hx}$ aplanadas y apiladas
$$\begin{align*}
  \boldsymbol{h}_t&=f(\boldsymbol{x}_t,\boldsymbol{h}_{t-1},\mathbf{W}_h)\\%
  \boldsymbol{o}_t&=g(\boldsymbol{h}_t,\mathbf{W}_o)%
\end{align*}$$
* Por la regla de la cadena:
$$\frac{\partial L}{\partial\mathbf{W}_h}%
  =\frac{1}{T}\sum_{t=1}^T\frac{\partial\ell(y_y,\boldsymbol{o}_t)}{\partial\mathbf{W}_h}%
  =\frac{1}{T}\sum_{t=1}^T%
  \frac{\partial\ell(y_y,\boldsymbol{o}_t)}{\partial\boldsymbol{o}_t}%
  \frac{\partial g(\boldsymbol{h}_t,\mathbf{W}_o)}{\partial\boldsymbol{h}_t}%
  \frac{\partial\boldsymbol{h}_t}{\partial\mathbf{W}_h}$$
* El último término puede expandirse como sigue:
$$\frac{\partial\boldsymbol{h}_t}{\partial\mathbf{W}_h}%
  =\frac{\partial f(\boldsymbol{x}_t,\boldsymbol{h}_{t-1},\mathbf{W}_h)}{\partial\mathbf{W}_h}%
  +\frac{\partial f(\boldsymbol{x}_t,\boldsymbol{h}_{t-1},\mathbf{W}_h)}{\partial\boldsymbol{h}_{t-1}}%
  \frac{\partial\boldsymbol{h}_{t-1}}{\partial\mathbf{W}_h}$$
* Si se sigue expandiendo recursivamente, se llega a:
$$\frac{\partial\boldsymbol{h}_t}{\partial\mathbf{W}_h}%
  \!=\!\frac{\partial f(\boldsymbol{x}_t,\boldsymbol{h}_{t-1},\mathbf{W}_h)}{\partial\mathbf{W}_h}%
  +\!\sum_{i=1}^{t-1}\!\left[\prod_{j=i+1}^t%
    \frac{\partial f(\boldsymbol{x}_j,\boldsymbol{h}_{j-1},\mathbf{W}_h)}{\partial\boldsymbol{h}_{j-1}}\right]\!%
  \frac{\partial f(\boldsymbol{x}_i,\boldsymbol{h}_{i-1},\mathbf{W}_h)}{\partial\mathbf{W}_h}$$
* **Coste temporal:** $\;O(T)$ por paso; $O(T^2)$ en total
* **Reducción del coste:** $\;$ suma de $K$ términos más recientes
* **Aplicación a secuencias largas:** $\;$ con ventanas no solapadas sucesivas, manteniendo el estado oculto de la RNN en batches sucesivos durante el entrenamiento (sin barajar subsecuencias)